In [ ]:
# =============================================================================
# 📚 AGILE EFFORT PREDICTION: RESEARCH-GRADE QUANTILE REGRESSION (FIXED)
# =============================================================================
# FIXES:
# 1. Solved "idf vector is not fitted" by forcing fit_transform before saving.
# 2. Implements Quantile Regression (Lower/Median/Upper) for confidence intervals.
# =============================================================================

import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, accuracy_score, confusion_matrix
import joblib
import warnings

# Settings
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
np.random.seed(42)

print("✅ System Ready: Initializing Research-Grade Effort Engine...")

# ==========================================
# 1. DATA LOADING & CLEANING
# ==========================================
print("⏳ Loading Data...")
# Ensure this CSV is uploaded to your notebook environment
df_raw = pd.read_csv('agile_event_stream_dataset_6k.csv')

# Filter: We only want standard Fibonacci points (Agile Standard)
valid_points = [1, 2, 3, 5, 8, 13]
df = df_raw[df_raw['Story_Point'].isin(valid_points)].copy()

# ==========================================
# 2. SPRINT2VEC (CONTEXT EMBEDDINGS)
# ==========================================
print("🛠️ Generating 'Sprint2Vec' Context Features...")

df['Creation_Date'] = pd.to_datetime(df['Creation_Date_Change'])
df = df.sort_values('Creation_Date')

# A. Momentum (Team Velocity in last 14 days)
df['team_velocity_14d'] = df.rolling('14D', on='Creation_Date')['Story_Point'].sum()

# B. Sprint Load (Active tickets at creation time)
df['sprint_load_7d'] = df.rolling('7D', on='Creation_Date')['Issue_ID'].count()

# C. Sprint Pressure (Points / Time)
df['days_remaining'] = 14 - df['days_since_sprint_start']
df['pressure_index'] = df['Story_Point'] / df['days_remaining'].clip(lower=0.5)

# ==========================================
# 3. TEXT EMBEDDINGS (THE FIX)
# ==========================================
print("🧠 Generating Text Embeddings...")

df['full_text'] = df['Title'].fillna('') + " " + df['Description'].fillna('')

# Initialize
tfidf = TfidfVectorizer(
    max_features=100,
    stop_words='english',
    ngram_range=(1, 2)
)

# 🚨 CRITICAL FIX: fit_transform MUST run here
# This calculates the vocabulary and IDF weights needed for the backend
text_emb = tfidf.fit_transform(df['full_text']).toarray()

# Verify it is fitted
if not hasattr(tfidf, 'idf_'):
    raise ValueError("❌ CRITICAL ERROR: TF-IDF did not fit! Check your data.")
else:
    print(f"   ✅ TF-IDF Fitted successfully. Vocab size: {len(tfidf.vocabulary_)}")

# Add to DataFrame
text_cols = [f"txt_{i}" for i in range(text_emb.shape[1])]
df_text = pd.DataFrame(text_emb, columns=text_cols, index=df.index)
df = pd.concat([df, df_text], axis=1)

# Categorical Encodings
le_type = LabelEncoder()
df['Type_Code'] = le_type.fit_transform(df['Type'].astype(str))

# ==========================================
# 4. PREPARE DATASETS
# ==========================================
features = ['sprint_load_7d', 'team_velocity_14d', 'pressure_index',
            'total_links', 'Type_Code'] + text_cols
target = 'Story_Point'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ==========================================
# 5. TRAIN QUANTILE REGRESSION (XGBOOST)
# ==========================================
print("\n🚀 Training Quantile Models (Pinball Loss)...")

def train_quantile(alpha):
    model = xgb.XGBRegressor(
        objective='reg:quantileerror',
        quantile_alpha=alpha,
        n_estimators=600,
        learning_rate=0.03,
        max_depth=5,
        subsample=0.8,
        n_jobs=-1,
        random_state=42
    )
    model.fit(X_train, y_train)
    return model

print("   - Training Lower Bound (10th Percentile)...")
model_lower = train_quantile(0.10)

print("   - Training Median (50th Percentile)...")
model_median = train_quantile(0.50)

print("   - Training Upper Bound (90th Percentile)...")
model_upper = train_quantile(0.90)

print("✅ Models Trained.")

# ==========================================
# 6. EVALUATION
# ==========================================
print("\n📊 Evaluating Performance...")

pred_raw = model_median.predict(X_test)
lower_bound = model_lower.predict(X_test)
upper_bound = model_upper.predict(X_test)

# Practical accuracy check (±1 Fibonacci level)
fibs = np.array([1, 2, 3, 5, 8, 13])
def snap_to_fib(val): return fibs[(np.abs(fibs - val)).argmin()]
pred_fib = [snap_to_fib(p) for p in pred_raw]
mae = mean_absolute_error(y_test, pred_raw)

print(f"   Mean Absolute Error: {mae:.2f} Points")
print(f"   Interval Coverage:   {np.mean((y_test >= lower_bound) & (y_test <= upper_bound))*100:.1f}%")

# ==========================================
# 7. SAVE ARTIFACTS (CRITICAL STEP)
# ==========================================
print("\n💾 Saving Artifacts for Main.py...")

# A. Save the 3 XGBoost Models
model_lower.save_model("effort_model_lower.json")
model_median.save_model("effort_model_median.json")
model_upper.save_model("effort_model_upper.json")

# B. Save the Preprocessors (The "Brain")
# Note: We save 'tfidf' which we CONFIRMED is fitted in Step 3
artifacts = {
    'tfidf': tfidf,
    'le_type': le_type
}
joblib.dump(artifacts, "effort_artifacts.pkl")

print("✅ SUCCESS! Download these 4 files and upload them to 'backend/models/':")
print("   1. effort_model_lower.json")
print("   2. effort_model_median.json")
print("   3. effort_model_upper.json")
print("   4. effort_artifacts.pkl")

✅ System Ready: Initializing Research-Grade Effort Engine...
⏳ Loading Data...
🛠️ Generating 'Sprint2Vec' Context Features...
🧠 Generating Text Embeddings...
   ✅ TF-IDF Fitted successfully. Vocab size: 100

🚀 Training Quantile Models (Pinball Loss)...
   - Training Lower Bound (10th Percentile)...
   - Training Median (50th Percentile)...
   - Training Upper Bound (90th Percentile)...
✅ Models Trained.

📊 Evaluating Performance...
   Mean Absolute Error: 0.34 Points
   Interval Coverage:   96.1%

💾 Saving Artifacts for Main.py...
✅ SUCCESS! Download these 4 files and upload them to 'backend/models/':
   1. effort_model_lower.json
   2. effort_model_median.json
   3. effort_model_upper.json
   4. effort_artifacts.pkl


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import joblib
import os

# 1. Load your dataset
csv_path = 'agile_event_stream_dataset_6k.csv'

if not os.path.exists(csv_path):
    print(f"❌ Error: Could not find '{csv_path}'. Please ensure it is in this folder.")
    exit()

print("⏳ Loading dataset...")
df_raw = pd.read_csv(csv_path)

# Filter for standard points to match your training logic
valid_points = [1, 2, 3, 5, 8, 13]
df = df_raw[df_raw['Story_Point'].isin(valid_points)].copy()

# Prepare Text (Title + Description)
df['full_text'] = df['Title'].fillna('') + " " + df['Description'].fillna('')

# 2. FIT TF-IDF (The Critical Fix)
print("🧠 Training TF-IDF Vectorizer...")
tfidf = TfidfVectorizer(
    max_features=100,
    stop_words='english',
    ngram_range=(1, 2)
)

# This line calculates the IDF weights. Without this, the model crashes.
tfidf.fit(df['full_text'])

# Check if it worked
if hasattr(tfidf, 'idf_'):
    print("   ✅ TF-IDF fitted successfully (Vocabulary & Weights present).")
else:
    print("   ❌ TF-IDF failed to fit.")
    exit()

# 3. FIT LabelEncoder
print("🏷️ Training Label Encoder...")
le_type = LabelEncoder()
le_type.fit(df['Type'].astype(str))

# 4. Save the Artifacts
output_file = "effort_artifacts.pkl"
artifacts = {
    'tfidf': tfidf,
    'le_type': le_type
}

joblib.dump(artifacts, output_file)
print(f"\n✅ SUCCESS! Generated '{output_file}'.")
print("👉 Now move this file to your backend/models/ folder and restart the server.")

⏳ Loading dataset...
🧠 Training TF-IDF Vectorizer...
   ✅ TF-IDF fitted successfully (Vocabulary & Weights present).
🏷️ Training Label Encoder...

✅ SUCCESS! Generated 'effort_artifacts.pkl'.
👉 Now move this file to your backend/models/ folder and restart the server.
